In [ ]:
%run -i ../python/common.py
publish=False

if not publish:
    # cleanup any old state
    # demke - fill in as we see what state gets generated in this page.
    bashCmds('''[[ -d mydir ]] && rm -rf mydir
    #''')
else:
    bashCmds('''rm -rf ~/*''')
    
closeAllOpenTtySessions()

import pandas as pd
#import seaborn as sns
import matplotlib.pyplot as plt

In [ ]:
appdir=os.getenv('HOME')
appdir=appdir + "/sync"
output = runTermCmd("[[ -d " + appdir + " ]] &&  rm -rf "+ appdir + 
             ";cp -r ../src/sync " + appdir)

bash = BashSession(cwd=appdir)

(cont:sync:scalability)=
# Scalability

*Scalability* refers to the ability of a system to handle a growing amount of work, for instance by increasing the number of threads (and processing cores) that are used to carry out the work. Writing scalable software requires us to think about the impact of synchronization because locking serializes the execution of critical sections, and thereby limits our ability to use multiple processors in parallel. In this chapter, we first briefly discuss the impact of [lock granularity](cont:sync:scalability:granularity) on scalability, look at how cache behavior affects performance on modern multiprocessors, and then describe strategies for cache-friendly locking.

(cont:sync:scalability:granularity)=
## Lock Granularity

*Lock granularity* refers to the amount of data that is protected by a single lock. In course-grained locking, there are a small number of locks, each protecting a large amount of data. An example of coarse-grained locking would be using a single lock to protect all the accounts in our bank server. In {numref}`name: fig:sync:scalability:granularity:coarse`, we illustrate the scalability issue in coarse-grained locking using the example of a hashtable with a single lock. In the figure, four threads are operating on the hash table; T0 has acquired the lock and T1, T2 and T3 are all waiting to acquire the lock. Assuming they are running on a multiprocessor with at least 4 cores, they can all run in parallel, but they are serialized by the hashtable lock. Regardless of how many threads (and cores) we add, the throughput of hashtable operations will be no better than if we had used a single thread. There will be high contention on the hashtable lock, and we will have poor scalability.  

```{figure} ../images/sync/coarsegrained-locking.drawio.png
---
width: 40%
name: fig:sync:scalability:granularity:coarse
---
Hashtable with coarse-grained locking. A single lock protects the entire hashtable, represented by the lightly shaded area.
```

To increase concurrency and reduce lock contention, we can increase the number of locks that are used, with each lock protecting a smaller amount of data. We mentioned this finer-grained locking strategy in {numref}`cont:sync:readmostly:rcu:why`, using the example of a hashtable with per-bucket locks. In {numref}`fig:sync:scalability:granularity:fine` we show how fine-grained locking improves concurrency for the hashtable. All the threads are accessing different buckets, so each can acquire a different lock, and all threads can operate on the hashtable in parallel. As long as the thread operations are uniformly distributed across the hash buckets, we will have much better scalability. 

```{figure} ../images/sync/finegrained-locking.drawio.png
---
width: 40%
name: fig:sync:scalability:granularity:fine
---
Hashtable with fine-grained locking. Each bucket in the hashtable has a separate lock, represented by the lightly shaded areas.
```

Of course, there are also usability tradeoffs to consider when deciding on the appropriate lock granularity. Coarse-grained locking is easy to reason about and reduces the risks of nasty concurrency bugs like deadlock or livelock. In the extreme, if there is only a single lock to acquire, there is no question about how to order multiple locks, or whether all the needed locks have been acquired. Fine-grained locking, on the other hand, means that threads may have to acquire multiple locks to complete an operation. It can become quite difficult to reason about the correctness of the locking strategy, and many subtle bugs are possible. In addition, there is some overhead to acquiring a lock, even without any contention, and more locks means more overhead. 

(cont:sync:scalability:caching)=
## Cache Effects

Designing our data structures for concurrent execution using fine-grained locking or lock-free operations does not guarantee good scalability, unfortunately. To understand why, we need to take a look at the architecture of a modern multicore multiprocessor system as shown in {numref}`fig:sync:scalability:cache:numa`. Today, most processors contain multiple compute cores on a single chip (i.e., a single processor is multicore), along with some fast per-core cache memory and some cache memory that is shared by all the cores on a chip (this is the last-level cache, called LLC if the number of cache levels is not specified, or L3 in {numref}`fig:sync:scalability:cache:numa`). The processor also contains a memory controller to communicate with DRAM memory connected to the chip, as well as inter-processor links, and I/O links. To scale the number of processors in a system, computer architects typically use a *non-uniform memory access* (NUMA) design,  Each multicore processor with its directly attached *local* DRAM memory forms a *node* in the NUMA system. Nodes are connected via the inter-processor communication links (QPI in Intel processors), which allow threads running on code on one node to access *remote* memory on any other node. This architecture is called NUMA because local memory is faster to access than remote memory. 

```{figure} ../images/sync/numa.drawio.png
---
width: 50%
name: fig:sync:scalability:cache:numa
---
Typical four-node NUMA system.
```

When we have multiple distinct caches, such as the L3 cache in each of the NUMA nodes, a *cache coherence protocol* ensures that all processors see a consistent view of memory. We're not going to get into all the details of these protocols, but there are a few important things to know. Data is loaded into a cache, or evicted from a cache in blocks called *lines*, with a typical line size being 64 bytes. That means a cache line could hold 16 4-byte integers, or 8 8-byte long integers, for example. When a processor issues a *load* for a memory location (e.g., because a thread executing on a core on that processor wants to use the value stored in an integer variable), it is loaded into the cache along with all the other memory locations on the same line. If threads on several different nodes need to read the same shared variable, the line holding that data item will be loaded into the L3 cache on each node in a *Shared* state. When a processor issues a *store* for a memory location, the coherence protocol invalidates any copies of the relevant cache line  in other nodes' caches, so that the writing processor can have an *Exclusive* copy in its cache. The store can only complete once the processor has an exclusive copy of the line. This is all carried out by the hardware in response to load and store instructions, without requiring any additional effort from software. Note that the same sort of coherence protocol is required to keep a consistent view of memory between the per-core caches within a single processor. 

### Shared Counter Example

```{figure} ../images/sync/sharedcounter_single.drawio.png
---
width: 40%
name: fig:sync:scalability:cache:sharedcounter_single
---
Each thread runs on a separate processor, performing some work and atomically incrementing a shared counter.
```

With that basic understanding of NUMA multiprocessors, let's take a look at the scalability of a simple operation: incrementing a shared counter using a hardware `atomic_inc` instruction. The experimental results in this section were obtained on an IBM S85 Enterprise Server PowerPC bus-based 24-way multiprocessor, packaged on four processor cards with six processors per card. This is an older architecture than that shown in {numref}`fig:sync:scalability:cache:numa`, but the main idea is similar. In our experiment, we have a varying number of threads that each repeatedly perform an operation that consists of some independent work followed by atomically incrementing a shared counter, as illustrated in {numref}`fig:sync:scalability:cache:sharedcounter_single`. On a uniprocessor, the atomic increment is roughly 0.1% of the total work per operation. Since each thread can do 99.9% of its work from each thread completely in parallel, we would expect that the average time per operation would stay roughly the same as we add more threads. Unfortunately, {numref}`fig:sync:scalability:cache:sharedcounter` reveals that this is not the case. The performance is 16% worse on two processors than on a single processor, and an order of magnitude worse at 24 processors.

```{figure} ../images/sync/sharedcounter_graph.drawio.png
---
width: 40%
name: fig:sync:scalability:cache:sharedcounter
---
Average time per operation as a function of number of threads incrementing a shared counter.
```

The culprit is contention in the memory system as all the processors compete to atomically increment the single shared counter variable. So, perhaps we can solve the problem by giving each processor its own counter to increment by allocating an array of counters. If each thread increments a different counter, they should be able to execute 100% in parallel. If we needed the final total, we could simply sum up all the per-thread counters at the end of the experiment. We illustrate this setup in {numref}`fig:sync:scalability:cache:sharedcounter_array`.

```{figure} ../images/sync/sharedcounter_array.drawio.png
---
width: 40%
name: fig:sync:scalability:cache:sharedcounter_array
---
Each thread runs on a separate processor, performing some work and atomically incrementing its own entry in an array of counters.
```

The performance of the array of counters is shown in {numref}`fig:sync:scalability:cache:sharedcounter_array_graph`. For the first 16 processors, the performance achieved using an array of per-thread counters is no better than when we had a single shared counter. With more than 16 processors, we see some improvement, but still far from ideal. 

```{figure} ../images/sync/sharedcounter_array_graph.drawio.png
---
width: 40%
name: fig:sync:scalability:cache:sharedcounter_array_graph
---
Average time per operation as a function of number of threads incrementing a shared counter (red) or an array of counters (blue).
```

So, what's going on? The problem is simply that adjacent array elements fall on the same cache line, and with a 64-byte line and 4-byte integer counters, 16 counters fit into one cache line. The extra coherence traffic between caches is exactly the same with the array of counters as it was with the single counter between 1 and 16 processors, because the unit of sharing on a multiprocessor is a cache line. When we have more than 16 processors, some of the counters land on a different cache line, and we are able to perform some of the increments in parallel because our threads are now accessing two cache lines instead of one.

We illustrate..

```{figure} ../images/sync/sharedcounter_array_cache.drawio.png
---
width: 100%
name: fig:sync:scalability:cache:sharedcounter_array_cache
---
Illustration of cache coherence traffic as multiple processors access an array of counters on the same cache line.
```

(cont:sync:scalability:cache-friendly)=
## Cache-friendly Locking